In [1]:
import pandas as pd
import pdfplumber as pdfp
import numpy as np
import os
import re
import calendar
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

In [2]:
def drop_unneeded(item):
    if '(' in item:
        return item[:item.find('(')]
    else:
        return item

            
def integrate_all_data(x):
    
    pdf_list = []
    for i in os.listdir('./'):
        if 'ipynb' not in i and '.pdf' not in i:
            pdf_list.append(i)
    
    lst = []
    for i in pdf_list:
        for j in os.listdir('./%s'%i.strip('.csv')+'/'):
                lst.append(j)
    
    
    company_list = np.unique(lst)
    
    integrated_list = []
    for i in company_list:
        integrated_list.append(i.strip('.csv')+'_integrated')
    
    for i in pdf_list:
        for j in os.listdir('./%s'%i+'/'):
            locals()[j.strip('.csv')+'_integrated'] = pd.DataFrame()
    
    for i in pdf_list:
        for j in range(len(integrated_list)):
            try:
                locals()[integrated_list[j]] = pd.concat([eval(integrated_list[j]),pd.read_csv('./%s'%i+'/%s'%company_list[j])]) 
            except:
                continue


    for i in integrated_list:
        locals()[i] = eval(i).rename(columns = {'Unnamed: 0':'Date'})
        locals()[i] = eval(i).set_index('Date')
        eval(i).index = eval(i).index.map(lambda x:str(x)).map(drop_unneeded)
        #locals()[i] = eval(i).set_index('Date')
        try: 
            eval(i).index = pd.to_datetime(eval(i).index,format = '%b-%y')
        except:
            try:
                eval(i).index = pd.to_datetime(eval(i).index,format = '%y-%b')
            except:
                try:
                    eval(i).index = pd.to_datetime(eval(i).index,format = '%m/%d/%Y')
                except:
                    continue
        eval(i).replace('nan',np.nan,inplace = True)
        locals()[i] = eval(i).groupby(eval(i).index.strftime('%Y-%m')).bfill().ffill()
        locals()[i] = eval(i)[~eval(i).index.duplicated(keep = 'first')]
        locals()[i] = eval(i).sort_index()
     
    new_folder = '../'+'integrated tables/'
    if not os.path.exists('../integrated tables'):
        os.makedirs(new_folder)
    if x == 'all':
        for i in integrated_list:
            if os.path.exists(new_folder+'%s.csv'%i):
                os.remove(new_folder+'%s.csv'%i)
                eval(i).to_csv(new_folder+'%s.csv'%i)
            else:
                eval(i).to_csv(new_folder+'%s.csv'%i)

    elif x in integrated_list:
        if os.path.exists(new_folder+'%s.csv'%x):
                os.remove(new_folder+'%s.csv'%x)
                eval(i).to_csv(new_folder+'%s.csv'%x)
        else:
            eval(i).to_csv(new_folder+'%s.csv'%x)

    else:
        return "wrong company's name. Please check again"
    #return(eval(x))
    
integrate_all_data(input("please enter the company's name: ")) # the format of the input is Company_Name_integrated, "all" for download them all

please enter the company's name:  all


In [23]:
lst = []
for root, dirs, files in os.walk('.'):
    lst.append(root)

def replace_item(item):
    if '.\\' in item:
        return item.replace('.\\','./')
    else:
        return item
lst = lst[2:]
pd.Series(lst).map(replace_item)

'./1Q18 Portfolio Company Valuations_vF'

In [8]:
a = pd.Series([np.nan,'3',np.nan,'4'])

a.bfill()

0    3
1    3
2    4
3    4
dtype: object